In [17]:
import urllib.request
from urllib.error import URLError, HTTPError, ContentTooShortError
import re
import itertools
from string import ascii_lowercase

In [19]:
def download(url,user_agent='wswp',tries=1):
    print('Downloading... {}'.format(url))
    request=urllib.request.Request(url)
    request.add_header('User-agent',user_agent)
    ctr=0
    while (ctr<tries):
        try:
            html=urllib.request.urlopen(url).read()
            return str(html)
        except (URLError, HTTPError, ContentTooShortError) as e:
            print('Error: {} (try: {})'.format(e.reason,ctr+1))
            if hasattr(e,'code') and 500<=e.code<600:
                ctr+=1
                continue
            break

In [24]:
from urllib.parse import urlparse
import time
class Throttle:
    """Add a delay between downloads to the same domain
    """
    def __init__(self, delay):
        # amount of delay between downloads for each domain
        self.delay = delay
        # timestamp of when a domain was last accessed
        self.domains = {}
    def wait(self, url):
        domain = urlparse(url).netloc
        last_accessed = self.domains.get(domain)
        if self.delay > 0 and last_accessed is not None:
            sleep_secs = self.delay - (time.time() - last_accessed)
            if sleep_secs > 0:
                # domain has been accessed recently
                # so need to sleep
                time.sleep(sleep_secs)
        # update the last accessed time
        self.domains[domain] = time.time()

In [63]:
throttle = Throttle(2)
a=[]
for x in ascii_lowercase:
    throttle.wait('https://money.rediff.com')
    html=str(download('https://money.rediff.com/companies/{}'.format(x)))
    len=re.search('>Showing 1 - (.*?) of (.*?) ',html)
    a.append(int(len.group(2)))

Downloading... https://money.rediff.com/companies/a
Downloading... https://money.rediff.com/companies/b
Downloading... https://money.rediff.com/companies/c
Downloading... https://money.rediff.com/companies/d
Downloading... https://money.rediff.com/companies/e
Downloading... https://money.rediff.com/companies/f
Downloading... https://money.rediff.com/companies/g
Downloading... https://money.rediff.com/companies/h
Downloading... https://money.rediff.com/companies/i
Downloading... https://money.rediff.com/companies/j
Downloading... https://money.rediff.com/companies/k
Downloading... https://money.rediff.com/companies/l
Downloading... https://money.rediff.com/companies/m
Downloading... https://money.rediff.com/companies/n
Downloading... https://money.rediff.com/companies/o
Downloading... https://money.rediff.com/companies/p
Downloading... https://money.rediff.com/companies/q
Downloading... https://money.rediff.com/companies/r
Downloading... https://money.rediff.com/companies/s
Downloading.

In [76]:
index=a

In [102]:
def search_codes(url):
    html=str(download(url))
    k=re.findall(r'<td>(\d\d\d\d\d\d)</td>',html)
    return k

In [103]:
throttle = Throttle(2)
ctr=0
b=[]
prod=[]
for x in ascii_lowercase:
    throttle.wait('https://money.rediff.com')
    for i in itertools.count(1,200):
        if(i>limit):
            break
        b=search_codes('https://money.rediff.com/companies/{}/{}-{}'.format(x,i,i+199))
        for item in b:
            prod.append(int(item))
        limit=index[ctr]
    ctr+=1
b=search_codes('https://money.rediff.com/companies/others')
for item in b:
    prod.append(int(item))

Downloading... https://money.rediff.com/companies/a/1-200
Downloading... https://money.rediff.com/companies/a/201-400
Downloading... https://money.rediff.com/companies/a/401-600
Downloading... https://money.rediff.com/companies/b/1-200
Downloading... https://money.rediff.com/companies/b/201-400
Downloading... https://money.rediff.com/companies/c/1-200
Downloading... https://money.rediff.com/companies/c/201-400
Downloading... https://money.rediff.com/companies/d/1-200
Downloading... https://money.rediff.com/companies/e/1-200
Downloading... https://money.rediff.com/companies/f/1-200
Downloading... https://money.rediff.com/companies/g/1-200
Downloading... https://money.rediff.com/companies/g/201-400
Downloading... https://money.rediff.com/companies/h/1-200
Downloading... https://money.rediff.com/companies/i/1-200
Downloading... https://money.rediff.com/companies/i/201-400
Downloading... https://money.rediff.com/companies/j/1-200
Downloading... https://money.rediff.com/companies/k/1-200
Do

In [114]:
import csv

In [122]:
with open('scripcodes.csv','w') as f:
    writer=csv.writer(f)
    writer.writerow(prod)
f.close()